In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "8"
import bbmaster as mst
import numpy as np
import matplotlib.pyplot as plt
import pymaster as nmt
import healpy as hp

### 0. Exciting polarization modes in maps
Producing polarized delta sims with `healpy` is slightly less trivial than for a scalar field, since setting the temperature - polarization cross correlation to zero actually suppresses the polarized mode. E.g. exciting $E$ therefore requires setting both $EE$ and $TE$ to one, which is fine as long as we don't care about $T$.

### 1. Setup
- Read mask.
- Define bandpower bins
- Initialize bandpower window function calculator

In [ ]:
# bins
bins_edges = np.loadtxt('data/bins.txt')
bins_edges_min = [] ; bins_edges_max = []
for n in range(len(bins_edges)-1):
	bins_edges_min.append(int(bins_edges[n]))
	bins_edges_max.append(int(bins_edges[n+1]))
bins = nmt.NmtBin.from_edges(bins_edges_min, bins_edges_max, is_Dell=False)

In [ ]:
# Read mask
nside = 64
msk = hp.read_map("data/mask_SAT_FirstDayEveryMonth_apo5.0_fpthin8_nside512.fits")
msk = hp.ud_grade(msk,nside)

# Generate NaMaster bins
#bins = nmt.NmtBin(nside, nlb=10)

# Bbl calculator
# Dictionary defining method to generate delta sims (not many options yet!)
#dsim = {'stats': 'Gaussian'}
#filt = {'mask': msk}
#bc = mst.DeltaBbl(nside, dsim, filt, b, pol=True, interp_ells=False, lmin=20, lmax=50)


### 2. Compute exact bandpowers
Now let's use NaMaster to compute the exact bandpower windows, and plot them

In [ ]:
# Spin-2 field
#map_E = np.ones(12*nside**2)
#almE = hp.map2alm(map_E,pol=False)
#map_tqu = hp.alm2map((np.zeros_like(almE,dtype=np.complex128),almE,np.zeros_like(almE,dtype=np.complex128)),nside,pol=True)

f = nmt.NmtField(hp.ud_grade(msk, nside_out=nside), None, spin=2)
#f = nmt.NmtField(hp.ud_grade(msk, nside_out=nside), map_tqu[1:], spin=2)
w = nmt.NmtWorkspace()
w.compute_coupling_matrix(f, f, bins)
mcm = w.get_coupling_matrix()

In [ ]:
def get_grid_axes():
    nbins=4
    fig, axes = plt.subplots(nbins, nbins,
                             figsize=(15, 10),
                             constrained_layout=True,
                             sharex=True)
    return fig, axes
# MCM has dimensions (pol_out, bpw_out, pol_in, ell_in)
lmax = 110 #3*nside-1
bpw_true = np.zeros((4,bins.get_n_bands(),4,lmax+1))
fig, axes = get_grid_axes()

for ip, p in enumerate(['EE', 'EB', 'BE', 'BB']):
    for iq, q in enumerate(['EE', 'EB', 'BE', 'BB']):
        ax = axes[iq, ip]
        # The assumed ordering of power spectra is such that the l-th element 
        # of the i-th power spectrum be stored with index l * n_cls + i.
        idx_p = [l*4+ip for l in range(lmax+1)]
        idx_q = [l*4+iq for l in range(lmax+1)]
        mcm_pq = mcm[np.ix_(idx_p,idx_q)]
        #print(mcm_pq.shape)
        #print(mcm_pq.T[0].shape)
        bpw_true[ip,:,iq,:] = np.array([bins.bin_cell(m) for m in mcm_pq.T]).T
        for ib in range(bins.get_n_bands()):
            ax.set_title(f'{q}->{p}')
            ax.plot(bpw_true[ip,ib,iq,:])
        #ax.set_xlim([0, 3*nside-1])
        ax.set_xlim([20, 110])
        ax.set_xlabel('$\ell$', fontsize=15)
        if ip==0:
            ax.set_ylabel('$B_{b\ell}$', fontsize=15)
        #ax.set_ylim(-0.11,0.11)
plt.show()

### 3. Compute bandpower windows through simulations

Now compute them using Z2 sims

In [ ]:
filt = {'mask': msk}
dsim = {'stats': 'Z2'}

In [ ]:
%%time

outdir = '/home/chervias/CMBwork/SimonsObs/BBMASTER/output/'
indir = '/home/chervias/CMBwork/SimonsObs/BBMASTER/output/'
# the order for the map_in is ell , seed, map
#in_name = '/map_tqu_nside64_%s_ell%04i_seed%i_map%i_NoBeam.fits'
#in_name = '/out_1week_%s_ell%04i_seed%i_map%i/filterbin_filtered_map.fits'

# mode 0=default, 1=only run until generating the maps and save them, 2=read in maps from disk and reconstruct bandpowers
# pure EE=1, BB=2

# This is for mode=1, to produce sims and save them
bc_z2_interp5 = mst.DeltaBbl(nside, dsim, filt, bins, pol=True, interp_ells=None, lmin=20, lmax=110, nsim_per_ell=100, save_maps= True, pure=0, mode=1, indir=indir, seed0=1000, outdir=outdir )
bpw_num_z2_interp5 = bc_z2_interp5.gen_Bbl_all()

#interp_ells = [20,25,28,30,32,35,40,43,45,50]
#bc_z2_interp5 = mst.DeltaBbl(nside, dsim, filt, bins, pol=True, interp_ells=5, lmin=20, lmax=50, nsim_per_ell=10, save_maps= False, pure=0, mode=2, indir=indir, in_name=in_name )
#bc_z2_interpCustom = mst.DeltaBbl(nside, dsim, filt, bins, pol=True, interp_ells=interp_ells, lmin=20, lmax=50, nsim_per_ell=10, save_maps= False, pure=0, mode=2, indir=indir, in_name=in_name )
#bpw_num_z2_interp5 = bc_z2_interp5.gen_Bbl_all()
#bpw_num_z2_interpCustom = bc_z2_interpCustom.gen_Bbl_all()
#bc_z2_all = mst.DeltaBbl(nside, dsim, filt, bins, pol=True, interp_ells=False, lmin=20, lmax=110, nsim_per_ell=10, save_maps= False, pure=0, mode=2, indir=indir, in_name=in_name )
#bpw_num_z2_all = bc_z2_all.gen_Bbl_all()

In [ ]:
%%time

indir = '/home/chervias/CMBwork/SimonsObs/BBMASTER/output/'
#in_name = '/out_1week_%s_ell%04i_seed%i_map%i/filterbin_filtered_map.fits' 
in_name = '/map_tqu_nside64_%s_ell%04i_seed%i_map%i_TOAST-obsmat-filtered.fits'
#in_name = '/map_tqu_nside64_%s_ell%04i_seed%i_map%i_filtered.fits'

bc_z2 = mst.DeltaBbl(nside, dsim, filt, bins, pol=True, interp_ells=1, lmin=20, lmax=110, nsim_per_ell=100, save_maps= False, pure=0, mode=2, indir=indir, in_name=in_name, seed0=1000 )
#bpw_num_z2 = bc_z2.gen_Bbl_all()

#interp_ells = [20,25,28,30,32,35,40,43,45,50]
#bc_z2_interpCustom = mst.DeltaBbl(nside, dsim, filt, bins, pol=True, interp_ells=interp_ells, lmin=20, lmax=50, nsim_per_ell=10, save_maps= False, pure=0, mode=2, indir=indir, in_name=in_name )
#bpw_num_z2_interpCustom = bc_z2_interpCustom.gen_Bbl_all()

In [ ]:
bpw_num_z2 = np.load('output/BPWFs_Z2_100sims_nside0064_TOAST-obsmat-filtered.npy')
#errors = np.load('output/Std_Z2_100sims_nside0064_TOAST-obsmat-filtered.npy')

In [ ]:
fig, axes = get_grid_axes()
for ip, p in enumerate(['EE', 'EB', 'BE', 'BB']):
    for iq, q in enumerate(['EE', 'EB', 'BE', 'BB']):
        ax = axes[iq, ip]
        # BPW matrices have dimensions (pol_out, bpw_out, pol_in, ell_in) 
        ax.set_title(f'{q}->{p}')    
        #for ib in range(bc_z2.n_bins):
        #for ib in [0]:
        for ib in range(1):
            #ax.plot(bc.get_ells(), bpw_num[iq,ib,ip,:], 'r-')
            #ax.plot(bc_z2_interp5.get_ells(), bpw_num_z2_interp5[iq,ib,ip,:], 'r-', label='100 sims')
            
            ax.plot(bc_z2.get_ells(), bpw_num_z2[iq,ib,ip,:], 'r:')
            #for l_sampled in bc_z2.errors.keys():
            #    ax.scatter(l_sampled,bpw_num_z2[iq,ib,ip,l_sampled-20],)
            #    ax.errorbar(l_sampled,bpw_num_z2[iq,ib,ip,l_sampled-20], yerr=bc_z2.errors[l_sampled][iq,ib,ip], ecolor='r',capsize=5)
            
            #ax.plot(bc_z2_interpCustom.get_ells(), bpw_num_z2_interpCustom[iq,ib,ip,:], 'b:')
            #for l_sampled in bc_z2_interpCustom.errors.keys():
            #    ax.errorbar(l_sampled,bpw_num_z2_interpCustom[iq,ib,ip,l_sampled-20],yerr=bc_z2_interpCustom.errors[l_sampled][iq,ib,ip],ecolor='b',capsize=5)
            
            #ax.plot(bc_z2_interp5.get_ells(), bpw_num_z2_interp5[iq,ib,ip,:], 'b-', label='100 sims')
            #for l_sampled in bc_z2_interp5.errors.keys():
            #    ax.errorbar(l_sampled,bpw_num_z2_interp5[iq,ib,ip,l_sampled-20],yerr=bc_z2_interp5.errors[l_sampled][iq,ib,ip],ecolor='b',capsize=5)
            
            #ax.plot(bpw_true[iq,ib,ip,:], 'k--',)
        #ax.set_xlim([20, 50])
        ax.set_xlim([20,110])
        ax.set_xlabel('$\ell$', fontsize=15)
        if ip==0:
            ax.set_ylabel('$B_{b\ell}$', fontsize=15)
        #ax.set_ylim(-0.11,0.11)
        #if iq==0 and ip==0:
        #    ax.legend(loc='best',fontsize=8)
plt.suptitle("Interpolate multipoles")
#plt.savefig('plots/Bandpowers_30-40_Test100sims.pdf',format='pdf',bbox_inches='tight')

In [ ]:
Nbins = 8
print( bins.get_ell_list(0)-20 )

In [ ]:
lmax+1-20

In [ ]:
# Fully binned coupling matrix (including filtering)
btbmcm = np.transpose(np.array([np.sum(bpw_num_z2[:, :, :, bins.get_ell_list(i)-20],axis=-1) for i in range(Nbins)]), axes=[1, 2, 3, 0])

# Invert and multiply by tbmcm to get final bandpower window functions.
ibtbmcm = np.linalg.inv(btbmcm.reshape([4*Nbins, 4*Nbins]))
winflat = np.dot(ibtbmcm, bpw_num_z2.reshape([4*Nbins, 4*(lmax+1-20)]))
wcal_inv = ibtbmcm.reshape([4, Nbins, 4, Nbins])
bpw_windows = winflat.reshape([4, Nbins, 4, lmax+1-20])

In [ ]:
# calculate the pseudoCL spectra for the 100 observed maps
#beam = hp.gauss_beam(np.radians(0.5), lmax=2000, pol=True)

# reshape w_inv
winv = wcal_inv.reshape([4*Nbins, 4*Nbins])

Nmc = 8
nside = 512
pwf = hp.pixwin(nside, pol=True, lmax=2000)
mask = hp.read_map("data/mask_SAT_FirstDayEveryMonth_apo5.0_fpthin8_nside512.fits")
#mask = hp.ud_grade(mask,nside)

PCell_obs = np.zeros((Nmc,4,Nbins))
for mm in range(Nmc):
    #map_ = hp.read_map('/home/chervias/CMBwork/SimonsObs/BBMASTER/output/CMB_sims/%04i/cmb_20231222_realistic_lensing_ns512_%04i_mcut-filtered.fits' % (mm,mm), field=(1,2))
    #map_ = hp.read_map('/home/chervias/CMBwork/SimonsObs/BBMASTER/output/CMB_sims/%04i/cmb_20231222_realistic_lensing_onlyBB_ns512_%04i_mcut-filtered.fits' % (mm,mm), field=(1,2))
    map_ = hp.read_map('/home/chervias/CMBwork/SimonsObs/BBMASTER/output/Validation_for_paper/CMBl-pureB_NoBeam/%04i/filterbin_coadd-full_map.fits' % (mm), field=(1,2))
    f2_map = nmt.NmtField(mask, map_, spin=2, purify_e=False, purify_b=True, beam=pwf[1])
    PCell = bins.bin_cell(nmt.compute_coupled_cell(f2_map, f2_map)[:,0:lmax+1])
    PCell_obs[mm] = np.dot(winv, PCell.flatten()).reshape([4, Nbins])
    print('Done for %i'%mm)

In [ ]:
cl_lens = np.loadtxt('/home/chervias/CMBwork/SimonsObs/BB_forecast/inputs/Cell_fiducial_DataChallenge_lensed_r0.dat') * (1e-6)**2 # transform to K^2 
#cl_tens = np.loadtxt('/home/chervias/CMBwork/SimonsObs/BB_forecast/inputs/Cell_fiducial_DataChallenge_tensor_r1.dat') * (1e-6)**2 # transform to K^2 
cl_th = np.array([cl_lens[:,2], np.zeros_like(cl_lens[:,2]), np.zeros_like(cl_lens[:,2]), cl_lens[:,3]])
#cl_dec_th = np.einsum('ijkl,kl', bpw_windows, cl_th[:,20:lmax+1])

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(1,1,1)

#ax.errorbar(bins.get_effective_ells(),np.mean(PCell_obs[:,0],axis=0), yerr=np.std(PCell_obs[:,0],axis=0),label='decoupled EE',fmt='o',markeredgecolor='red',markerfacecolor='none',ecolor='red',capsize=5,markeredgewidth=1,elinewidth=1)
ax.errorbar(bins.get_effective_ells(),np.mean(PCell_obs[:,3],axis=0), yerr=np.std(PCell_obs[:,3],axis=0),label='decoupled BB',fmt='o',markeredgecolor='blue',markerfacecolor='none',ecolor='blue',capsize=5,markeredgewidth=1,elinewidth=1)

#ax.plot(np.arange(lmax+1),cl_lens[:lmax+1,2],c='grey',label='CMB EE theory',lw=1,ls='--',)
ax.plot(np.arange(lmax+1), cl_lens[:lmax+1,3],c='grey',label='CMB BB theory (lensing)',lw=1,ls='--',)

#ax.scatter(bins.get_effective_ells(), cl_dec_th[0], label='theory EE', marker='^', edgecolors='red', facecolors='none')
#ax.scatter(bins.get_effective_ells(), cl_dec_th[3], label='theory BB', marker='^', edgecolors='blue', facecolors='none')

#ax.plot(cl_lens[:,2],c='b')

ax.set_yscale('log')
#ax.set_xscale('log')
ax.set_xlim(30,100)
#ax.set_ylim(1e-19,1e-17)
ax.set_ylabel('C_ell K^2')
ax.legend()

#plt.savefig('plots/Cells_reconstructed_CMBlensing_mcut.pdf', format='pdf', bbox_inches='tight')

In [ ]:
np.mean(PCell_obs[:,3],axis=0)

In [ ]:
lmax = 110 #3*nside - 1
#mcm_byme_all = np.zeros_like(mcm)
#mcm_byme_every5 = np.zeros_like(mcm)
mcm_byme_interp5 = np.zeros_like(mcm)

ell_array = np.arange(20,lmax+1)
ell_sampled = np.array([20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100,105,110])

#for ip in range(4):
#    for iq in range(4):
#        for i_ell_1 in range(lmax+1):
#            for i_ell_2 in range(lmax+1):
#                mcm_byme_every5[i_ell_1*4 + ip , i_ell_2*4 + iq ] = bc_z2_interp5.cb_ell[ip,iq,i_ell_1,i_ell_2]

for ip in range(4):
    for iq in range(4):
        for ell in range(20,lmax+1):
            bc_z2_interp5.cb_ell[ip,iq,20:,ell] = np.interp(ell_array, ell_sampled, bc_z2_interp5.cb_ell[ip,iq,ell_sampled,ell],left=0.0,right=0.0)

for ip in range(4):
    for iq in range(4):
        for i_ell_1 in range(lmax+1):
            for i_ell_2 in range(lmax+1):
                mcm_byme_interp5[i_ell_1*4 + ip , i_ell_2*4 + iq ] = bc_z2_interp5.cb_ell[ip,iq,i_ell_1,i_ell_2]

#for ip in range(4):
#    for iq in range(4):
#        for i_ell_1 in range(lmax+1):
#            for i_ell_2 in range(lmax+1):
#                mcm_byme_all[i_ell_1*4 + ip , i_ell_2*4 + iq ] = bc_z2_all.cb_ell[ip,iq,i_ell_1,i_ell_2]

In [ ]:
fig = plt.figure(figsize=(15,5))

ax = fig.add_subplot(1,3,1)
ax.set_title('coupling matrix exact')
ax.matshow(mcm,vmin=-0.001,vmax=0.001)

ax = fig.add_subplot(1,3,2)
ax.set_title('coupling matrix from MC sims, every ell')
ax.matshow(mcm_byme_all,vmin=-0.001,vmax=0.001)

ax = fig.add_subplot(1,3,3)
ax.set_title('coupling matrix from MC sims, every 5 ells interp')
ax.matshow(mcm_byme_interp5,vmin=-0.001,vmax=0.001)


Note that when setting `interp_ells=5` (only sample every 5th multipole and interpolate the others), we clearly see that Z2 variables are superior to Gaussian variables at low $\ell$.

Now, let's bin a fiducial spectra with this bandpower and compare that to an observed map

In [ ]:
# now load a map and calculate its deconvolved power spectra
msk = hp.read_map("data/mask_SAT_1week_apo10.0_fpthin8_nside512.fits")
#m_ = hp.read_map('toast/output/%04i/test1_1week_pureE_seed%04i/filterbin_filtered_map.fits'%(0,0),field=(0,1,2))
m_ = hp.read_map('output/CMB_sims/cmb_20230608_pureB_ns512_%04i.fits'%(0),field=(0,1,2))
f_s2 = nmt.NmtField(msk, [m_[1],m_[2]] , spin=2)

w_new = nmt.NmtWorkspace()
w_new.compute_coupling_matrix(f_s2, f_s2, bins)
w_new.update_coupling_matrix(mcm_byme_interp5)

Nmc = 100
cl_decoupled = np.zeros((Nmc,4,bins.get_n_bands()))
for m in range(Nmc):
    #m_ = hp.read_map('toast/output/%04i/test1_1week_pureE_seed%04i/filterbin_filtered_map.fits'%(m,m),field=(0,1,2))
    m_ = hp.read_map('output/CMB_sims/cmb_20230608_pureB_ns512_%04i.fits'%(m),field=(0,1,2))
    f_s2 = nmt.NmtField(msk, [m_[1],m_[2]] , spin=2)
    cl_coupled = nmt.compute_coupled_cell(f_s2, f_s2)
    cl_decoupled[m] = w_new.decouple_cell(cl_coupled)


In [ ]:
fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(1,1,1)

ell_array = np.arange(4000+1)
cl_lens = np.loadtxt('/home/chervias/CMBwork/SimonsObs/BB_forecast/inputs/Cell_fiducial_DataChallenge_lensed_r0.dat') # transform to K^2 
ax.plot(ell_array,cl_lens[0:4000+1,3]*(1e3)**2,c='black',lw=0.6,label='theory BB r=0 (only lensing)')
#ax.plot(ell_array,cl_lens[0:4000+1,2]*(1e3)**2,c='black',lw=0.6,label='theory EE')

ells = bins.get_effective_ells()

ax.errorbar(ells[0:7],np.mean(cl_decoupled[:,0,0:7]*(1e9)**2,axis=0),yerr=np.std(cl_decoupled[:,0,0:7]*(1e9)**2,axis=0),label='reconstructed EE',fmt='o',markeredgecolor='blue',markerfacecolor='none',ecolor='blue',capsize=5)
ax.errorbar(ells[0:7],np.mean(cl_decoupled[:,1,0:7]*(1e9)**2,axis=0),yerr=np.std(cl_decoupled[:,1,0:7]*(1e9)**2,axis=0),label='reconstructed EB',fmt='o',markeredgecolor='green',markerfacecolor='none',ecolor='green',capsize=5)
ax.errorbar(ells[0:7],np.mean(cl_decoupled[:,2,0:7]*(1e9)**2,axis=0),yerr=np.std(cl_decoupled[:,2,0:7]*(1e9)**2,axis=0),label='reconstructed BE',fmt='o',markeredgecolor='orange',markerfacecolor='none',ecolor='orange',capsize=5)
ax.errorbar(ells[0:7],np.mean(cl_decoupled[:,3,0:7]*(1e9)**2,axis=0),yerr=np.std(cl_decoupled[:,3,0:7]*(1e9)**2,axis=0),label='reconstructed BB',fmt='o',markeredgecolor='red',markerfacecolor='none',ecolor='red',capsize=5)

ax.set_xlim(30,200)
ax.set_ylim(-0.5,4.2)
#ax.set_ylim(-30,700)
#ax.set_yscale('symlog',linthresh=1e-19)
#ax.set_yscale('log')
#ax.set_xscale('log')
ax.set_xlabel('multipole')
ax.set_ylabel(r'$\hat{C}_{\ell}$ [nK$^2_{\rm CMB}$]')
ax.legend(loc='upper right')

plt.savefig('plots/Cells_reconstructed_pureB_onlyMask.pdf',format='pdf',bbox_inches='tight')

In [ ]:
(Npols,Nbins,Npols,Nells) = bpw_num_z2.shape

In [ ]:
cl_lens = np.loadtxt('/home/chervias/CMBwork/SimonsObs/BB_forecast/inputs/Cell_fiducial_DataChallenge_lensed_r0.dat') * (1e-6)**2 # transform to K^2 
#pwf_64 = hp.pixwin(64,pol=True)
#pwf_512 = hp.pixwin(512,pol=True)

l_min = 20

# we will only calculate the tilde(C)^EE
C_b_EE = np.zeros(Nbins)
C_b_BB = np.zeros(Nbins)

for b in range(Nbins):
    #if b in [3,4,5,6,7,8]:
    if b in [0,1,2,3,4,5]:
        for q in range(Npols):
            for ell in range(Nells):
                C_b_EE[b] += bpw_num_z2[0,b,q,ell]*cl_lens[ell+l_min,2]  # 2 is EE
                C_b_BB[b] += bpw_num_z2[3,b,q,ell]*cl_lens[ell+l_min,3]  # 3 is BB

In [ ]:
msk = hp.read_map("data/mask_SAT_1week_apo10.0_fpthin8_nside512.fits")
Nmc = 2

Cell_filt_ee = np.zeros((Nmc,6,3*nside))
for m in range(Nmc):
    m_ = hp.read_map('toast/output/%04i/test1_1week_pureE_seed%04i/filterbin_filtered_map.fits'%(m,m),field=(0,1,2))
    cl = hp.anafast(m_ * msk, pol=True, lmax=(3*nside-1),iter=5)
    #print(cl.shape)
    Cell_filt_ee[m] = cl

Cell_filt_bb = np.zeros((Nmc,6,3*nside))
for m in range(Nmc):
    m_ = hp.read_map('toast/output/%04i/test1_1week_pureB_seed%04i/filterbin_filtered_map.fits'%(m,m),field=(0,1,2))
    cl = hp.anafast(m_ * msk, pol=True, lmax=(3*nside-1),iter=5)
    #print(cl.shape)
    Cell_filt_bb[m] = cl

In [ ]:
fig = plt.figure()

ax = fig.add_subplot(1,1,1)

ells = bins.get_effective_ells()

for m in range(Nmc):
    #ax.scatter(ells, Dell_filt[m,0], color='blue')
    if m==0:
        ax.plot(Cell_filt_ee[m,1],c='blue',lw=0.05,label='anafast over pure E map')
        ax.plot(Cell_filt_bb[m,2],c='red',lw=0.05,label='anafast over pure B map')
    else:
        ax.plot(Cell_filt_ee[m,1],c='blue',lw=0.05)
        ax.plot(Cell_filt_bb[m,2],c='red',lw=0.05)

ax.scatter(ells,C_b_EE,color='blue',label='theory EE binned by Bbl')
ax.scatter(ells,C_b_BB,color='red',label='theory BB binned by Bbl')


ax.set_xlim(20,100)
ax.set_yscale('log')
ax.set_xlabel('multipole')
ax.set_ylabel(r'$\tilde{C}_{\ell}$')
ax.legend(loc='lower right')

Now, let's test other bandpower widths.

In [ ]:
b20 = nmt.NmtBin(nside, nlb=20)
# Bbl calculator
dsim = {'stats': 'Gaussian'}
filt = {'mask': msk}
bc20 = mst.DeltaBbl(nside, dsim, filt, b20, pol=True)

In [ ]:
w20 = nmt.NmtWorkspace()
w20.compute_coupling_matrix(f, f, b20)
mcm20 = w20.get_coupling_matrix()

# MCM has dimensions (pol_out, bpw_out, pol_in, ell_in) 
bpw20_true = np.zeros((4,bc20.n_bins,4,3*nside))

for ip, p in enumerate(['EE', 'EB', 'BE', 'BB']):
    for iq, q in enumerate(['EE', 'EB', 'BE', 'BB']):
        ax = axes[iq, ip]
        # The assumed ordering of power spectra is such that the l-th element 
        # of the i-th power spectrum be stored with index l * n_cls + i.
        idx_p = [l*4+ip for l in range(3*nside)]
        idx_q = [l*4+iq for l in range(3*nside)]
        mcm_pq = mcm20[np.ix_(idx_p,idx_q)]
        bpw20_true[ip,:,iq,:] = np.array([b20.bin_cell(m) for m in mcm_pq.T]).T

In [ ]:
%%time
bpw20_num = bc20.gen_Bbl_all()

In [ ]:
%%time
dsim = {'stats': 'Z2'}
bc20_z2 = mst.DeltaBbl(nside, dsim, filt, b20, pol=True)
bpw20_num_z2 = bc20_z2.gen_Bbl_all()

In [ ]:
fig, axes = get_grid_axes()

for ip, p in enumerate(['EE', 'EB', 'BE', 'BB']):
    for iq, q in enumerate(['EE', 'EB', 'BE', 'BB']):
        ax = axes[iq, ip]
        # BPW matrices have dimensions (pol_out, bpw_out, pol_in, ell_in) 
        ax.set_title(f'{q}->{p}')    
        for ib in range(bc20.n_bins):
            ax.plot(bc20.get_ells(), bpw20_num[iq,ib,ip,:], 'r-')
            ax.plot(bc20_z2.get_ells(), bpw20_num_z2[iq,ib,ip,:], 'c:')
            ax.plot(bpw20_true[iq,ib,ip,:], 'k--')
        ax.set_xlim([0, 3*nside-1])
        ax.set_xlabel('$\ell$', fontsize=15)
        if ip==0:
            ax.set_ylabel('$B_{b\ell}$', fontsize=15)
plt.suptitle("Vary multipole bins")
plt.show()

In [ ]:
# Next steps:
# * Only sample fraction of ells and interpolate to gain speed  OK
# * Vary bandpowers & check                                     OK
# * Implement chi2 measure of bandpower accuracy                TODO
# * Load test CMB signal and reconstruct observed bandpowers    TODO
# * Implement r likelihood and make Fisher forecast             TODO